# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 2 2023 9:11AM,258515,20,ATR-I000104,"HVL, LLC dba Atrium Innovations",Released
1,Mar 2 2023 9:06AM,258514,10,CLI00481.1,"CLINUVEL, Inc.",Released
2,Mar 2 2023 9:04AM,258513,10,CTF0011664,"Citieffe, Inc.",Released
3,Mar 2 2023 9:04AM,258512,19,9224133,VITABIOTICS USA INC.,Released
4,Mar 2 2023 9:01AM,258511,19,9224131,VITABIOTICS USA INC.,Released
5,Mar 2 2023 9:00AM,258510,19,9224132,VITABIOTICS USA INC.,Released
6,Mar 2 2023 8:57AM,258508,19,9224130,VITABIOTICS USA INC.,Released
7,Mar 2 2023 8:57AM,258509,10,SO94554,"Senseonics, Incorporated",Released
8,Mar 2 2023 8:52AM,258507,12,HH-40293,Hush Hush,Released
9,Mar 2 2023 8:52AM,258507,12,HH-40294,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,258511,Released,1
22,258512,Released,1
23,258513,Released,1
24,258514,Released,1
25,258515,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
258511,NaN,1.0
258512,NaN,1.0
258513,NaN,1.0
258514,NaN,1.0
258515,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
258441,0.0,1.0
258448,1.0,0.0
258478,1.0,0.0
258483,0.0,1.0
258487,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
258441,0,1
258448,1,0
258478,1,0
258483,0,1
258487,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,258441,0,1
1,258448,1,0
2,258478,1,0
3,258483,0,1
4,258487,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,258441,,1
1,258448,1,
2,258478,1,
3,258483,,1
4,258487,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 2 2023 9:11AM,258515,20,"HVL, LLC dba Atrium Innovations"
1,Mar 2 2023 9:06AM,258514,10,"CLINUVEL, Inc."
2,Mar 2 2023 9:04AM,258513,10,"Citieffe, Inc."
3,Mar 2 2023 9:04AM,258512,19,VITABIOTICS USA INC.
4,Mar 2 2023 9:01AM,258511,19,VITABIOTICS USA INC.
5,Mar 2 2023 9:00AM,258510,19,VITABIOTICS USA INC.
6,Mar 2 2023 8:57AM,258508,19,VITABIOTICS USA INC.
7,Mar 2 2023 8:57AM,258509,10,"Senseonics, Incorporated"
8,Mar 2 2023 8:52AM,258507,12,Hush Hush
28,Mar 2 2023 8:46AM,258506,10,"Digital Brands, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 2 2023 9:11AM,258515,20,"HVL, LLC dba Atrium Innovations",,1
1,Mar 2 2023 9:06AM,258514,10,"CLINUVEL, Inc.",,1
2,Mar 2 2023 9:04AM,258513,10,"Citieffe, Inc.",,1
3,Mar 2 2023 9:04AM,258512,19,VITABIOTICS USA INC.,,1
4,Mar 2 2023 9:01AM,258511,19,VITABIOTICS USA INC.,,1
5,Mar 2 2023 9:00AM,258510,19,VITABIOTICS USA INC.,,1
6,Mar 2 2023 8:57AM,258508,19,VITABIOTICS USA INC.,,1
7,Mar 2 2023 8:57AM,258509,10,"Senseonics, Incorporated",,1
8,Mar 2 2023 8:52AM,258507,12,Hush Hush,,20
9,Mar 2 2023 8:46AM,258506,10,"Digital Brands, LLC",,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 2 2023 9:11AM,258515,20,"HVL, LLC dba Atrium Innovations",1,
1,Mar 2 2023 9:06AM,258514,10,"CLINUVEL, Inc.",1,
2,Mar 2 2023 9:04AM,258513,10,"Citieffe, Inc.",1,
3,Mar 2 2023 9:04AM,258512,19,VITABIOTICS USA INC.,1,
4,Mar 2 2023 9:01AM,258511,19,VITABIOTICS USA INC.,1,
5,Mar 2 2023 9:00AM,258510,19,VITABIOTICS USA INC.,1,
6,Mar 2 2023 8:57AM,258508,19,VITABIOTICS USA INC.,1,
7,Mar 2 2023 8:57AM,258509,10,"Senseonics, Incorporated",1,
8,Mar 2 2023 8:52AM,258507,12,Hush Hush,20,
9,Mar 2 2023 8:46AM,258506,10,"Digital Brands, LLC",2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 2 2023 9:11AM,258515,20,"HVL, LLC dba Atrium Innovations",1,
1,Mar 2 2023 9:06AM,258514,10,"CLINUVEL, Inc.",1,
2,Mar 2 2023 9:04AM,258513,10,"Citieffe, Inc.",1,
3,Mar 2 2023 9:04AM,258512,19,VITABIOTICS USA INC.,1,
4,Mar 2 2023 9:01AM,258511,19,VITABIOTICS USA INC.,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 2 2023 9:11AM,258515,20,"HVL, LLC dba Atrium Innovations",1.0,NaN
1,Mar 2 2023 9:06AM,258514,10,"CLINUVEL, Inc.",1.0,NaN
2,Mar 2 2023 9:04AM,258513,10,"Citieffe, Inc.",1.0,NaN
3,Mar 2 2023 9:04AM,258512,19,VITABIOTICS USA INC.,1.0,NaN
4,Mar 2 2023 9:01AM,258511,19,VITABIOTICS USA INC.,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 2 2023 9:11AM,258515,20,"HVL, LLC dba Atrium Innovations",1.0,0.0
1,Mar 2 2023 9:06AM,258514,10,"CLINUVEL, Inc.",1.0,0.0
2,Mar 2 2023 9:04AM,258513,10,"Citieffe, Inc.",1.0,0.0
3,Mar 2 2023 9:04AM,258512,19,VITABIOTICS USA INC.,1.0,0.0
4,Mar 2 2023 9:01AM,258511,19,VITABIOTICS USA INC.,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2843502,30.0,3.0
12,516994,20.0,1.0
19,1809481,18.0,6.0
20,517014,2.0,0.0
22,516946,1.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2843502,30.0,3.0
1,12,516994,20.0,1.0
2,19,1809481,18.0,6.0
3,20,517014,2.0,0.0
4,22,516946,1.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,30.0,3.0
1,12,20.0,1.0
2,19,18.0,6.0
3,20,2.0,0.0
4,22,1.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,30.0
1,12,Released,20.0
2,19,Released,18.0
3,20,Released,2.0
4,22,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,20,22
Status,,,,,
Executing,3.0,1.0,6.0,0.0,1.0
Released,30.0,20.0,18.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,20,22
0,Executing,3.0,1.0,6.0,0.0,1.0
1,Released,30.0,20.0,18.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,20,22
0,Executing,3.0,1.0,6.0,0.0,1.0
1,Released,30.0,20.0,18.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()